# Classification with TensorFlow Neural Decision Forests with TFX MLOps

## The scenario

Predicting sepsis from patient data, but this time with automation. The key benefit of machine learning operations comes from the automation of the model life cycle steps. When new training data becomes available, a workflow which includes data validation, preprocessing, model training, analysis, and deployment can be triggered.

## The automation

Tensorflow Extended (TFX).

## The data

This example uses synthetically generated patient data containing biological markers. The task is binary classification to predict whether a patient is septic or not based on bio-markers. The dataset includes 100,000 instances with 4 numerical features (Heart Rate, Respiratory Rate, Temperature, and White Blood Cell Count) labelled with 0 (not septic) or 1 (septic).

For more details review the data_file folder.

## The model

TensorFlow Decision Forests (TF-DF).

# Configuration

In [1]:
## Install packages and libraries

import tempfile
import tensorflow as tf
import urllib.request
import os
import pandas as pd
import shutil
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_decision_forests as tfdf

from absl import logging
from pathlib import Path
from tfx import v1 as tfx
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.proto import example_gen_pb2

print(f"Tensorflow Version: {tf.__version__}")
print(f"TFX Version: {tfx.__version__}")
print(f"TFDF Version: {tfdf.__version__}")

print(f"Tensorflow Data Validation Version: {tfdv.__version__}")

logging.set_verbosity(logging.INFO)

2022-08-19 22:11:36.715653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 22:11:36.715709: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Tensorflow Version: 2.9.1
TFX Version: 1.9.1
TFDF Version: 0.2.7
Tensorflow Data Validation Version: 1.9.0


## Configure Variables

In [2]:
PIPELINE_NAME = "mlops"
MODEL_NAME = "base"

TRAIN_DATA = '../data/training_data'
DATA_SOURCE_PATH = Path(TRAIN_DATA) / 'septic_data_labelled.csv'

SAVED_DATA = '../data/transformed_data'
DATA_TRAIN_FILENAME = Path(SAVED_DATA) / 'train.csv'
DATA_TEST_FILENAME = Path(SAVED_DATA) / 'test.csv'

# Output directory to store artifacts generated from the pipeline.
PIPELINE_DIRECTORY = os.path.join(Path('../pipelines'), PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('../pipelines/metadata', PIPELINE_NAME, 'metadata.db')

SCHEMA_DIRECTORY = os.path.join(PIPELINE_DIRECTORY, 'schema')
SCHEMA_FILE_NAME = str(os.path.join(SCHEMA_DIRECTORY, 'schema.pbtxt'))

# Output directory where created models from the pipeline will be exported.
MODEL_DIRECTORY = os.path.join('../models', MODEL_NAME)

# Module Paths
CONSTANTS_MODULE_PATH = '../src/constants.py'
TRANSFORM_MODULE_PATH = '../src/transform.py'
TRAINER_MODULE_PATH = '../src/trainer.py'

In [3]:
print(DATA_SOURCE_PATH)
print(DATA_TRAIN_FILENAME)
print(DATA_TEST_FILENAME)
print(MODEL_DIRECTORY)
print(PIPELINE_DIRECTORY)
print(METADATA_PATH)

../data/training_data/septic_data_labelled.csv
../data/transformed_data/train.csv
../data/transformed_data/test.csv
../models/base
../pipelines/mlops
../pipelines/metadata/mlops/metadata.db


## Load the data 

In [4]:
data_df = pd.read_csv(DATA_SOURCE_PATH)

# splitting the data for training and testing
data_df = data_df.sample(frac=1)
train_df = data_df[: int(len(data_df) * 0.7)]
test_df = data_df[int(len(data_df) * 0.7): ]

# removing the undesired columns from all the datasets
datasets = [train_df, test_df]
#drop_columns = ['User ID']
#for dataset in datasets:
#    dataset.drop(drop_columns, axis=1, inplace=True)

train_df.to_csv(DATA_TRAIN_FILENAME, index=False)
test_df.to_csv(DATA_TEST_FILENAME, index=False)

In [5]:
# peeking at the data
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 56157 to 91862
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Temp      100000 non-null  float64
 1   HR        100000 non-null  float64
 2   Resp      100000 non-null  float64
 3   WBC       100000 non-null  float64
 4   isSeptic  100000 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 4.6 MB


In [6]:
# peeking at the data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 56157 to 35202
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Temp      70000 non-null  float64
 1   HR        70000 non-null  float64
 2   Resp      70000 non-null  float64
 3   WBC       70000 non-null  float64
 4   isSeptic  70000 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 3.2 MB


## Define Common Functions

In [7]:
# creating the useful conversion functions

#def _bytes_feature(value):
#    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(str(value), encoding="raw_unicode_escape")]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [8]:
# This function will be required to convert out test set 
# to compatible schema types for inferencing
def _examples(df):
    examples = []
    for index, row in df.iterrows():
        features = {
            "Temp": _floats_feature(row['Temp']),
            "HR": _floats_feature(row['HR']),
            "Resp": _floats_feature(row['Resp']),
            "WBC": _floats_feature(row['WBC']),
            "isSeptic": _int64_feature(row['isSeptic']),
        }
        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        examples.append(example_proto.SerializeToString())

    return examples

## Interact with Pipeline

In [9]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.

from tfx.orchestration.experimental.interactive.interactive_context import (InteractiveContext)

context = InteractiveContext(pipeline_root=PIPELINE_DIRECTORY)

# Development & Training

## Ingest Data

The `ExampleGen` TFX component can ingest a few data structures, including comma- separated value files (CSVs), precomputed TFRecord files, and serialization outputs from Apache Avro and Apache Parquet.

- The CsvExampleGen component from the tfx will be used to ingest the csv data.
- The train data will be split into train-eval set in the ration of 3:1.

In [10]:
output = tfx.proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
                                                     tfx.proto.SplitConfig.Split(name="train", hash_buckets=3),
                                                     tfx.proto.SplitConfig.Split(name="eval", hash_buckets=1)
    ]))
example_gen = tfx.components.CsvExampleGen(input_base=SAVED_DATA, output_config=output)

context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ../data/transformed_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Check the Data with Statistics

The `StatisticsGen` component will be used to generate the statistics of the features present in the data.

These statistics help us in understanding the characteristics of the data. It recieves the example outputs generated by the previous generator component and generates the stats for each splits of the data.

In [11]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen, enable_cache=True)
context.show(statistics_gen.outputs['statistics'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to ../pipelines/mlops/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to ../pipelines/mlops/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


## Generate a schema for data

Data schema are a form of describing the representation of your data‐ sets. The schema definition of your dataset can then be used to validate future datasets to determine if they are in line with your previous training sets.

The easiest way to get the schema for the dataset is to infer it using the SchemaGen component.

In [12]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen, enable_cache=True)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to ../pipelines/mlops/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'HR',FLOAT,required,,-
'Resp',FLOAT,required,,-
'Temp',FLOAT,required,,-
'WBC',FLOAT,required,,-
'isSeptic',INT,required,,-


## Updating the schema

We have inferred the types and features in the dataset, but we can make the required changes in the schema as well.

The two environments that we need to create are:

- TRAINING: This environment will be used to validate the data for the training process.
- SERVING: This environment will be used to validate the data for the model in production.

We will remove the isSeptic (target feature) from the serving environment.

In [13]:
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, "schema.pbtxt")
)

# adding the needed environments
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# removing the `isSeptic` column from the serving environment
tfdv.get_feature(schema, "isSeptic").not_in_environment.append("SERVING")

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'HR',FLOAT,required,,-
'Resp',FLOAT,required,,-
'Temp',FLOAT,required,,-
'WBC',FLOAT,required,,-
'isSeptic',INT,required,,-


## Load the Schema

The importer node will be used to load a saved schema into the pipeline for subsequent components.

In [14]:
!mkdir -p {SCHEMA_DIRECTORY}

tfdv.write_schema_text(schema, SCHEMA_FILE_NAME)

# loading the updated schema using the importer node.
schema_importer = tfx.dsl.Importer(
    source_uri=str(SCHEMA_DIRECTORY),
    artifact_type=tfx.types.standard_artifacts.Schema
).with_id("schema_importer")

context.run(schema_importer, enable_cache=True)
context.show(schema_importer.outputs['result'])

INFO:absl:Running driver for schema_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: ../pipelines/mlops/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for schema_importer
INFO:absl:Running publisher for schema_importer
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'HR',FLOAT,required,,-
'Resp',FLOAT,required,,-
'Temp',FLOAT,required,,-
'WBC',FLOAT,required,,-
'isSeptic',INT,required,,-


## Validate the data

The data validation step checks that the data in your pipelines is what your feature engineering step expects. It assists you in comparing multiple datasets. It also highlights if your data changes over time, for example, if your training data is significantly different from the new data provided to your model for inference.

Data validation performs three distinct checks:
- Check for data anomalies.
- Check that the data schema hasn’t changed.
- Check that the statistics of our new datasets still align with statistics from our previous training datasets.

In [15]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']
)

context.run(example_validator, enable_cache=True)
context.show(example_validator.outputs['anomalies'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to ../pipelines/mlops/ExampleValidator/anomalies/5/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to ../pipelines/mlops/ExampleValidator/anomalies/5/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


## Transform the data

Data Transformation can be used to standardize some steps in the machine learning pipeline.

- Preprocessing your data efficiently in the context of the entire dataset
- Scaling the preprocessing steps effectively
- Avoiding a potential training-serving skew
- The transform component will be used here which makes sure that the same pre-processing is done on the training as well as the serving data.

We need to implement a preprocessing_fn() in order to do the necessary transformations.

In [16]:
%%writefile {CONSTANTS_MODULE_PATH}
LABEL = 'isSeptic'

Writing ../src/constants.py


In [17]:
%%writefile {TRANSFORM_MODULE_PATH}

import tensorflow as tf
import tensorflow_transform as tft

import constants

LABEL = constants.LABEL

def preprocessing_fn(inputs):

    outputs = dict()
    outputs['Temp'] = inputs['Temp']
    outputs['HR'] = inputs['HR']
    outputs['Resp'] = inputs['Resp']
    outputs['WBC'] = inputs['WBC']
    # example converting the `Gender` into label encoded column.
    # outputs['Gender'] = tf.cast(tf.equal(inputs['Gender'], 'male'), tf.int64)

    outputs[LABEL] = inputs[LABEL]

    return outputs

Writing ../src/transform.py


Creating the transform component.

In [18]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=os.path.abspath(TRANSFORM_MODULE_PATH),
)
context.run(transform, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-basic/src/transform.py' (including modules: ['transform', 'constants']).
INFO:absl:User module package has hash fingerprint version b4b99ce628d3f55a49913e414a19016776a3169475ceec64c3eb86f50f9e4858.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmp11dtdlz6/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp1z07_8z1', '--dist-dir', '/tmp/tmp545mlfc6']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying constants.py -> build/lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmp1z07_8z1
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmp1z07_8z1
copying build/lib/constants.py -> /tmp/tmp1z07_8z1
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '../pipelines/mlops/_wheels/tfx_user_code_Transform-0.0+b4b99ce628d3f55a49913e414a19016776a3169475ceec64c3eb86f50f9e4858-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@../pipelines/mlops/_wheels/tfx_user_code_Transform-0.0+b4b99ce628d3f55a49913e414a19016776a3169475ceec64c3eb86f50f9e4858-py3-none-any.whl'
INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@../pipelines/mlops/_wheels/tfx_user_code_Transform-0.0+b4b99ce628d3f55a49913e414a19016776a3169475ceec64c3eb86f50f9e4858-py3-none-any.whl', 'preprocessing_fn': None} 'preprocessing_fn'
INFO:absl:Installing '../pipelines/mlops/_wheels/tfx_user_code_Transform-0.0+b4b99ce628d3f55a4

...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

We can take a look at the transformed examples from our data.

In [19]:
train_uri = os.path.join(
    transform.outputs['transformed_examples'].get()[0].uri,
    'Split-train'
)

tfrecord_filenames = [
                      os.path.join(train_uri, name) for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)


features {
  feature {
    key: "HR"
    value {
      float_list {
        value: 200.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      float_list {
        value: 27.0
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 36.79999923706055
      }
    }
  }
  feature {
    key: "WBC"
    value {
      float_list {
        value: 10.979999542236328
      }
    }
  }
  feature {
    key: "isSeptic"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "HR"
    value {
      float_list {
        value: 166.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      float_list {
        value: 16.0
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 37.5
      }
    }
  }
  feature {
    key: "WBC"
    value {
      float_list {
        value: 3.549999952316284
      }
    }
  }
  feature {
    key: "isSeptic"
    value {
      int64_list {
  

## Train a model

To train the model we need to create a trainer component.

The Trainer component will look for a run_fn() function in our module file and use the function as an entry point to execute the training process.

The transformations needed to be included as part of the default signature of the model.

In [20]:
%%writefile {TRAINER_MODULE_PATH}

import tensorflow as tf
import tensorflow_decision_forests as tfdf
import tensorflow_transform as tft

from absl import logging
from tensorflow.keras import layers, Model, optimizers, losses, metrics
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from typing import List, Text

import constants

LABEL = constants.LABEL
BATCH_SIZE = 32
EPOCHS = 50

def _input_fn(
    file_pattern: List[Text],
    data_accessor: tfx.components.DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    batch_size: int,
) -> tf.data.Dataset:

    """
    Generates a dataset of features that can be used to train
    and evaluate the model.

    Args:
        file_pattern: List of paths or patterns of input data files.
        data_accessor: An instance of DataAccessor that we can use to
            convert the input to a RecordBatch.
        tf_transform_output: The transformation output.
        batch_size: The number of consecutive elements that we should
            combine in a single batch.

    Returns:
        A dataset that contains a tuple of (features, indices) where 
            features is a dictionary of Tensors, and indices is a single
            Tensor of label indices.
    """

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
        schema=tf_transform_output.raw_metadata.schema,
    )

    tft_layer = tf_transform_output.transform_features_layer()

    def apply_transform(raw_features):
        transformed_features = tft_layer(raw_features)
        transformed_label = transformed_features.pop(LABEL)
        return transformed_features, transformed_label
    
    return dataset.map(apply_transform).repeat()

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """
    Returns a function that parses a serialized tf.Example and applies
    the transformations during inference.
    Args:
        model: The model that we are serving.
        tf_transform_output: The transformation output that we want to 
            include with the model.
    """
    
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")])
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        
        required_feature_spec = {
            k: v for k, v in feature_spec.items() if k != LABEL
        }

        parsed_features = tf.io.parse_example(
            serialized_tf_examples,
            required_feature_spec
        )

        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn

def _model() -> tf.keras.Model:
    inputs = [
              layers.Input(shape=(1,), name="HR"),
              layers.Input(shape=(1,), name="Resp"),
              layers.Input(shape=(1,), name="Temp"),
              layers.Input(shape=(1,), name="WBC")
    ]

    x = layers.concatenate(inputs)
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=optimizers.Adam(1e-2),
        loss="binary_crossentropy",
        metrics=[metrics.BinaryAccuracy()],
    )

    model.summary(print_fn=logging.info)
    return model

def run_fn(fn_args: tfx.components.FnArgs):
    """
    The callback function that will be called by the Trainer component
    to train the model using the suplied arguments.

    Args:
        fn_args: A collection of name/value pairs representing the 
            arguments to train the model.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=BATCH_SIZE,
    )

    model = _model()
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        epochs=EPOCHS
    )

    # We need to modify the default signature to include the transform layer in 
    # the computational graph.
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(model, tf_transform_output),
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Writing ../src/trainer.py


We can now add a trainer component to train a model.

In [21]:
trainer = tfx.components.Trainer(
    examples=example_gen.outputs["examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
    module_file=os.path.abspath(TRAINER_MODULE_PATH),
)
context.run(trainer, enable_cache=False)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/mlops-basic/src/trainer.py' (including modules: ['transform', 'trainer', 'constants']).
INFO:absl:User module package has hash fingerprint version f39b5ba6e1294c469dbbe8b9e58078bc8abe206e43a779e66601003c8ed4503a.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpptkvppvm/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp2tt_zv8l', '--dist-dir', '/tmp/tmpzudwd_1l']


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying trainer.py -> build/lib
copying constants.py -> build/lib


/opt/app-root/lib64/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


installing to /tmp/tmp2tt_zv8l
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmp2tt_zv8l
copying build/lib/trainer.py -> /tmp/tmp2tt_zv8l
copying build/lib/constants.py -> /tmp/tmp2tt_zv8l
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt


INFO:absl:Successfully built user code wheel distribution at '../pipelines/mlops/_wheels/tfx_user_code_Trainer-0.0+f39b5ba6e1294c469dbbe8b9e58078bc8abe206e43a779e66601003c8ed4503a-py3-none-any.whl'; target user module is 'trainer'.
INFO:absl:Full user module path is 'trainer@../pipelines/mlops/_wheels/tfx_user_code_Trainer-0.0+f39b5ba6e1294c469dbbe8b9e58078bc8abe206e43a779e66601003c8ed4503a-py3-none-any.whl'
INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'train_args': '{\n  "num_steps": 100\n}', 'eval_args': '{\n  "num_steps": 5\n}', 'module_file': None, 'run_fn': None, 'trainer_fn': None, 'custom_config': 'null', 'module_path': 'trainer@../pipelines/mlops/_wheels/tfx_user_code_Trainer-0.0+f39b5ba6e1294c469dbbe8b9e58078bc8abe206e43a7

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature HR has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Resp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Temp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature WBC has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature isSeptic has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl: Layer (type)                   Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl: HR (InputLayer)                [(None, 1)]          0           []                               
INFO:absl:                                                                                   

Epoch 1/50
100/100 [==============================] - 2s 8ms/step - loss: 0.4619 - binary_accuracy: 0.9172 - val_loss: 0.2504 - val_binary_accuracy: 0.9312
Epoch 2/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2315 - binary_accuracy: 0.9388 - val_loss: 0.1802 - val_binary_accuracy: 0.9563
Epoch 3/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2409 - binary_accuracy: 0.9309 - val_loss: 0.2248 - val_binary_accuracy: 0.9500
Epoch 4/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2234 - binary_accuracy: 0.9356 - val_loss: 0.2246 - val_binary_accuracy: 0.9312
Epoch 5/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2312 - binary_accuracy: 0.9331 - val_loss: 0.2036 - val_binary_accuracy: 0.9375
Epoch 6/50
100/100 [==============================] - 1s 6ms/step - loss: 0.2367 - binary_accuracy: 0.9259 - val_loss: 0.2513 - val_binary_accuracy: 0.9187
Epoch 7/50
100/100 [==============================] - 1s 6ms/ste

INFO:tensorflow:Assets written to: ../pipelines/mlops/Trainer/model/7/Format-Serving/assets
INFO:absl:Training complete. Model written to ../pipelines/mlops/Trainer/model/7/Format-Serving. ModelRun written to ../pipelines/mlops/Trainer/model_run/7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Evaluate the model

Since we are done with creating and training a model, we need to evaluate its performance before we can deploy it.
The performance can be checked using the Evaluator component.

We are going to look at the val_accuracy of the model in this example, from two perspectives:

- For the overall dataset.

In [22]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
                 tfma.ModelSpec(
                     signature_name="serving_default",
                     preprocessing_function_names=['tft_layer'],
                     label_key="isSeptic",
                 )
    ],
    slicing_specs=[
                   tfma.SlicingSpec()
    ]
)

We want to deploy only when our model is outperforming the previous models and that model is termed as blessed model.

Resolver node is used to compare the current model with the previous one and deploy the blessed one.

In [23]:
model_resolver = tfx.dsl.Resolver(
    strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessings=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
).with_id("latest_blessed_model_resolver")

context.run(model_resolver)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessings: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessings, additional_properties={}, additional_custom_properties={}))

In [24]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    eval_config=eval_config,
    baseline_model=model_resolver.outputs["model"],
)

context.run(evaluator, enable_cache=False)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "model_specs": [\n    {\n      "label_key": "isSeptic",\n      "preprocessing_function_names": [\n        "tft_layer"\n      ],\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  signature_name: "serving_default"
  label_key: "isSeptic"
  preprocessing_function_names: "tft_layer"
}
slicing_specs {
}

INFO:absl:Using ../pipelines/mlops/Trainer/model/7/Form

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "model_specs": [\n    {\n      "label_key": "isSeptic",\n      "preprocessing_function_names": [\n        "tft_layer"\n      ],\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {}\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  signature_name: "serving_default"
  label_key: "isSeptic"
  preprocessing_function_names: "tft_layer"
}
slicing_specs {
}

INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_

INFO:absl:Evaluation complete. Results written to ../pipelines/mlops/Evaluator/evaluation/9.
INFO:absl:No threshold configured, will not validate model.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [27]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

total 0


## Push the model

The model will be pushed if the model is blessed.

In [25]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=MODEL_DIRECTORY)))
context.run(pusher, enable_cache=True)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model on ../pipelines/mlops/Evaluator/blessing/9 was not blessed by model validation
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

In [26]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

OSError: SavedModel file does not exist at: ../pipelines/mlops/Pusher/pushed_model/10/{saved_model.pbtxt|saved_model.pb}

## Inferencing

Now we can run the inferences on the pushed model.

In [ ]:
def get_inference_fn(MODEL_DIRECTORY):
    model_directories = (d for d in os.scandir(MODEL_DIRECTORY) if d.is_dir())
    model_path = max(model_directories, key=lambda i: int(i.name)).path

    loaded_model = tf.keras.models.load_model(model_path)

    return loaded_model.signatures["serving_default"]

In [ ]:
inference_fn = get_inference_fn(MODEL_DIRECTORY)
result = inference_fn(examples=tf.constant(_examples(test_df)))
print(result["output_0"].numpy())

In [ ]:
!cd .. && pwd && ls